In [ ]:
#default_exp training

# Imports

In [ ]:
#export
from tqdm.auto import tqdm
from speechsep.core import *
from speechsep.masks import *
from speechsep.pipe import *
from speechsep.utils import *
from speechsep.data import *
from speechsep.imports import *
from speechsep.models import *

In [ ]:
#export
from torch.utils.data import Dataset, DataLoader
import torch
import torchvision

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Enable Cuda

In [ ]:
if torch.cuda.is_available():
    torch.cuda.set_device(0)

# File Paths

In [ ]:
#export
#fn = Path("../../data/clips/")
fn = Path("../../datawav/")
pipe = AudioPipe(fn)

# Dataset/DataLoader

In [ ]:
#export
class Tensorify(Transform):
    def encodes(self, x, cuda=True):
        tnsr = complex2real(x.data) if hasattr(x, "data") else complex2real(x)
        return torch.cuda.FloatTensor(tnsr)

class AudioDataset(Dataset):
    @delegates(AudioPipe)
    def __init__(self, fn, **kwargs):
        self.fn = fn
        self.pipe = AudioPipe(fn, **kwargs)
        self.n_samples = len(get_audio_files(fn))
    
    def __getitem__(self, index):
        x,y = self.pipe(index)
        x,y = Tensorify()(x),Tensorify()(y)
        return x,y
    
    def __len__(self):
        return self.n_samples

# Loss Function

In [ ]:
#export
def loss_func(x,y):
    loss = nn.MSELoss()
    min_loss = min([loss(x[i], y[i]) for i in range(len(x))])
    return min_loss

# Training Loop

In [ ]:
#export
bs = 4
shuffle=True
workers=0

In [ ]:
#export
dataset = AudioDataset(fn)
n = len(dataset)
train_ds, valid_ds, test_ds = torch.utils.data.random_split(dataset, [5000, 2500, n-7500])
train_dl = DataLoader(dataset=train_ds, batch_size=bs, shuffle=shuffle, num_workers=workers)
valid_dl = DataLoader(dataset=valid_ds, batch_size=bs, shuffle=shuffle, num_workers=workers)
test_dl = DataLoader(dataset=test_ds, batch_size=bs, shuffle=shuffle, num_workers=workers)

dataiter = iter(train_dl)
data = dataiter.next()

In [ ]:
#export
model = U_Net(img_ch=2, output_ch=4).cuda()
model.train();

In [ ]:
#export
n_epochs = 10
n_samples = len(train_dl.dataset)
n_iter = math.ceil(n_samples/bs)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
#export
for epoch in range(n_epochs):
    for i, (xb, yb) in tqdm(enumerate(train_dl)):
        out = model(xb)
        mask1 = MaskcIRM(out[:,:2,:,:])
        mask2 = MaskcIRM(out[:,2:,:,:])
        sep = mask1*xb, mask2*xb
        loss = loss_func(sep, yb)

        #if (i+1)%5==0:
        #    print(f'epoch {epoch}: step {(i+1)/n_iter}')
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    torch.save(model.state_dict(), f"../models/unet_{epoch+1}")
    print(f"epoch {epoch+1} Finished, saved model")